In [1]:
import librosa
import librosa.display
import os
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random
import keras
# import datetime as dt
from datetime import datetime
import time
# import datetime.datetime



from keras import layers
from keras import models
# from keras import optimizers
from tensorflow.keras import optimizers

from keras.applications.densenet import DenseNet121, DenseNet169
#from keras.applications.nasnet import NASNetLarge
# from keras_efficientnets import EfficientNetB7
from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints
from keras.layers.normalization import BatchNormalization 
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from keras import backend as K

tf.set_random_seed(1234)
random.seed(100)

os.environ["CUDA_VISIBLE_DEVICES"]="1"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)


/usr/local/lib/python3.6/dist-packages/numba/errors.py:137: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)typ

In [2]:

def score_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

def dot_product(x, kernel):
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    
def get_unique_classes(input_directory,files):

    unique_classes=set()
    for f in files:
        g = f.replace('.mat','.hea')
        input_file = os.path.join(input_directory,g)
        with open(input_file,'r') as f:
            for lines in f:
                if lines.startswith('#Dx'):
                    tmp = lines.split(': ')[1].split(',')
                    for c in tmp:
                        unique_classes.add(c.strip())
    return sorted(unique_classes)

def one_hot_encoding(one_hot_vector,y, class2index):
    ind=class2index[y]
    one_hot_vector[ind]=1
    return one_hot_vector

# Search for multi-label subjects
def searching_overlap(input_directory,class2index, input_file_names):
    multiclasses=[]
    multisubjects=[]
    number = []
    for file in input_file_names:
        f=file
        g = f.replace('.mat','.hea')
        input_file = os.path.join(input_directory,g)
        with open(input_file,'r') as f:
            for lines in f:
                if lines.startswith('#Dx'):
                    tmp = lines.split(': ')[1].split(',')
                    if len(tmp)>1:
                        one_hot_vector = [0]*(len(class2index))
                        for c in tmp:
                            one_hot_vector = one_hot_encoding(one_hot_vector, c.strip(), class2index)
                        multiclasses.append(one_hot_vector)
                        multisubjects.append(g)
                        number.append(len(tmp))
    return multisubjects, multiclasses, number

def block_feature(sequence_en, minimum_len): 
    new_en = []
    if len(sequence_en) > minimum_len:  # 길이가 minimum보다 긴 경우
        start = random.randint(0,len(sequence_en)-minimum_len)
        #print(start)
        new_en = sequence_en[start:start+minimum_len]
    elif len(sequence_en) == minimum_len: # 길이가 minimum
        new_en = sequence_en
    else: 
        assert len(sequence_en) <= minimum_len
    return new_en

def exploratory_look(input_directory,file, class2index):
    classes = []
    f = file
    g = f.replace('.mat','.hea')
    input_file = os.path.join(input_directory,g)
    with open(input_file,'r') as f:
        for lines in f:
            if lines.startswith('#Dx'):
                tmp = lines.split(': ')[1].split(',')
                print(tmp, len(tmp))
    return tmp     

# Get classes of sorted file names
def get_labels(input_directory,file, class2index):
    f = file
    g = f.replace('.mat','.hea')
    input_file = os.path.join(input_directory,g)
    with open(input_file,'r') as f:
        for lines in f:
            if lines.startswith('#Dx'):
                tmp = lines.split(': ')[1].split(',')
                one_hot_vector = [0]*(len(class2index))
                for c in tmp:
                    one_hot_vector = one_hot_encoding(one_hot_vector, c.strip(), class2index)
                
    return one_hot_vector

def randextract_mels(curr_step, batch_size, data, mel_directory, class2index, minimum_len, x_mean_final, x_std_final):
    mel_files = []
    classes = []
    start = batch_size*curr_step
    end = batch_size*(curr_step+1)
    curr_file_indices = data[start:end]
    for file in curr_file_indices:
        tmp_file = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
        clip_file = block_feature(tmp_file, minimum_len)
        #print(clip_file.shape)
        #clip_file = tmp_file[:minimum_len]
        clip_file -= x_mean_final
        clip_file /= x_std_final
        mel_files.append(clip_file)
        label = get_labels(input_directory, file, class2index)
        classes.append(label)
    
    # YJS added for ABN -> should calculate 2 losses
#     classes_abn = [classes,classes]
    
    concat = list(zip(mel_files, classes))
#     concat = list(zip(mel_files, classes_abn))
    random.shuffle(concat)
    mel_files, classes = zip(*concat)
    return mel_files, classes

def train(data_train, mel_directory, batch_size, class2index, minimum_len, x_mean_final, x_std_final): 
    loss=[]
#     acc = []
    f1 = []

    total_steps = int(np.ceil(len(data_train)/batch_size))
    for curr_step in range(total_steps):
        batch_mels, batch_labels = randextract_mels(curr_step, batch_size, data_train, mel_directory, class2index, minimum_len, x_mean_final, x_std_final)
        batch_mels = np.asarray(batch_mels)
        batch_labels2 = np.asarray(np.squeeze(batch_labels))
        batch_labels = [batch_labels2, batch_labels2]
#         print(batch_labels.shape)
#         print(batch_labels)

# return of train
# 0 = total loss (attention branch + perception branch)
# 1 = loss of attention pred
# 2 = loss of perception pred
# 3 = f1 of attention pred
# 4 = f1 of perception pred     

        train_tmp = model.train_on_batch(batch_mels, batch_labels)
#         print(train_tmp)
        loss_ = train_tmp[0]/2
        f1_ = np.mean((train_tmp[3], train_tmp[4]))
        loss.append(loss_)
#         acc.append(train_tmp[1])
        f1.append(f1_)

    loss = np.mean(np.array(loss))
#     acc = np.mean(np.array(acc))
    f1 = np.mean(np.array(f1))
    return loss, f1

def test(data, mel_directory, input_directory, class2index, minimum_len, model, x_mean_final, x_std_final):
    scores = []
    predicted_labels=[]
    accuracy=np.zeros(len(data))
    #total_loss=[]
    total_acc = 0
    total_f1 = 0
    
    mel_files = []
    classes = []
    for i, file in enumerate(data):
        tmp_file = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
        steps = int(np.floor(tmp_file.shape[0]/minimum_len))
        mel_files = []
        for block in range(steps): # 128개씩 쪼갠 블럭 단위로 predict
            start = block*minimum_len
            end = (block+1)*minimum_len
            clip_file = tmp_file[start:end]
            clip_file -= x_mean_final
            clip_file /= x_std_final
            mel_files.append(clip_file)
        mel_files = np.asarray(mel_files)
        logit = model.predict(mel_files)
#         print(len(logit))
        logit = np.mean(logit, axis=0)
        logit = np.mean(logit, axis=0)
#         print(logit)
        pred = np.argmax(logit)
#         print('Pred={}'.format(pred))
        
        label = np.argmax(get_labels(input_directory, file, class2index))
#         print('Label={}'.format(label))
        #f1 = f1_score(label, logit)
        #print(pred, label)
        if pred == label:
            acc = 1
        else:
            acc = 0
        total_acc += acc
        #total_f1 += f1
    final_acc = total_acc / i
    #final_f1 = total_f1 / i
    return final_acc#, final_f1

batch_size = 32#16#20#32#5#2#1#10#32

minimum_len = 2880
epochs = 1000
loss_function = 'categorical_crossentropy'
activation_function = 'softmax'
rootdir = '../'
date = datetime.today().strftime("%Y%m%d")
input_directory = os.path.join(rootdir, 'Training_WFDB')
mel_name = 'Raw_data_20200424' 
mel_directory = os.path.join(rootdir, mel_name)
results_directory = os.path.join(rootdir, 'results_'+date+'_0_ieee_model')
if not os.path.isdir(input_directory):
    os.mkdir(input_directory)
if not os.path.isdir(mel_directory):
    os.mkdir(mel_directory)
if not os.path.isdir(results_directory):
    os.mkdir(results_directory)
        
input_files = []
for f in os.listdir(input_directory):
    if os.path.isfile(os.path.join(input_directory, f)) and not f.lower().startswith('.') and f.lower().endswith('mat'):
        input_files.append(f)
input_file_names = sorted(input_files)

unique_classes = get_unique_classes(input_directory, input_files)
class2index = {}
for a, b in enumerate(unique_classes):
    class2index[b] = a
    
classes_orig= [x.replace('.mat', '.hea') for x in input_file_names] # total subjects
classes_multi, _, _ = searching_overlap(input_directory,class2index, input_file_names)
classes_single = [x for x in classes_orig if x not in classes_multi]
classes_single = [x.replace('.hea', '.mat') for x in classes_single]

# double-checking if classes_single have single-label
a, b, c  = searching_overlap(input_directory,class2index,classes_single)

# we can safely use classes_single as input_file_names
input_file_names = classes_single
random.shuffle(input_file_names)
np.shape(input_file_names)

x_mean_all = []
x_std_all = []
for file in input_file_names:
    x = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
    x_mean = [np.mean(x[:,0]), np.mean(x[:,1]), np.mean(x[:,2]), np.mean(x[:,3]), np.mean(x[:,4]), np.mean(x[:,5]),
             np.mean(x[:,6]), np.mean(x[:,7]), np.mean(x[:,8]), np.mean(x[:,9]), np.mean(x[:,10]), np.mean(x[:,11])]
    
    x_std = [np.std(x[:,0]), np.std(x[:,1]), np.std(x[:,2]), np.std(x[:,3]), np.std(x[:,4]), np.std(x[:,5]),
             np.std(x[:,6]), np.std(x[:,7]), np.std(x[:,8]), np.std(x[:,9]), np.std(x[:,10]), np.std(x[:,11])]
    #print(x_mean)
    x_mean_all.append(x_mean)
    x_std_all.append(x_std) # yjs corrected on 2020-05-25
x_mean_final = np.mean(x_mean_all, axis=0)
x_std_final = np.mean(x_std_all, axis=0)
print(x_mean_final)

data, data_test = train_test_split(input_file_names, test_size = 0.2, train_size = 0.8, shuffle=True)
data_train, data_val = train_test_split(data, test_size = 0.25, train_size = 0.75, shuffle=True)
print(np.shape(data_train), np.shape(data_val), np.shape(data_test))


main_input = Input(shape=(minimum_len,12), dtype='float32', name='main_input')

branch_pred = []

[-2.24290386e-03 -4.58280585e-05  4.31697309e-03 -3.00174693e-03
 -2.36609229e-04  1.28997408e-03  2.17347589e-04 -7.99152384e-04
 -3.42993744e-03 -1.69711686e-03  1.27138164e-03  1.94670545e-03]
(3840,) (1280,) (1281,)



In [ ]:
# from tensorflow.keras.layers import GaussianNoise

# GaussianNoise(0.01)


In [3]:
from ABNmodules import *


model = get_custom_model((None, 12), 9, out_ch=256, n=18)
# model = get_model((None, 12), 9, n=7)
model.summary()
model.compile(loss=[loss_function, loss_function],
              optimizer=optimizers.Adam(lr=1e-5),
              
              
              
              metrics=[score_f1])



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, None, 12)]   0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, None, 64)     2368        input_image[0][0]                
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, 64)     256         conv1d[0][0]                     
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, None, 64)     12352       batch_norm

In [4]:
# DISP DATETIME FOR CHECKING TIME
start = time.time()
val_acc_sum=[]
train_loss_sum=[]
train_acc_sum=[]
val_loss_sum=[]
val_acc_min = 0
print(results_directory)

for num_epoch in range(epochs):
    random.shuffle(data_train)
    train_loss, train_f1 = train(data_train, mel_directory, batch_size, class2index, minimum_len, x_mean_final, x_std_final)
    print('\nEpoch',num_epoch+1,'train_loss:',f'{train_loss:.3f}','train_f1:',f'{train_f1:.3f}',"\t")
    model_output = "ECG_ABN_E%02dL%.2f" % (num_epoch, train_loss)
    save_name = os.path.join(results_directory, model_output)
    
    val_acc = test(data_val, mel_directory, input_directory, class2index, minimum_len, model, x_mean_final, x_std_final)

    if val_acc > val_acc_min:
        val_acc_min = val_acc
        model.save(save_name)
    print('\nValidation', num_epoch+1, 'valid_acc:',f'{val_acc:.3f}', 'best_acc:',f'{val_acc_min:.3f}', "\t")
print(time.time() - start)

../results_20200526_0_ieee_model

Epoch 1 train_loss: 2.645 train_f1: 0.033 	

Validation 1 valid_acc: 0.145 best_acc: 0.145 	

Epoch 2 train_loss: 2.188 train_f1: 0.031 	

Validation 2 valid_acc: 0.157 best_acc: 0.157 	

Epoch 3 train_loss: 2.159 train_f1: 0.036 	

Validation 3 valid_acc: 0.178 best_acc: 0.178 	

Epoch 4 train_loss: 2.155 train_f1: 0.042 	

Validation 4 valid_acc: 0.205 best_acc: 0.205 	

Epoch 5 train_loss: 2.124 train_f1: 0.042 	

Validation 5 valid_acc: 0.257 best_acc: 0.257 	

Epoch 6 train_loss: 2.109 train_f1: 0.044 	

Validation 6 valid_acc: 0.270 best_acc: 0.270 	

Epoch 7 train_loss: 2.096 train_f1: 0.049 	

Validation 7 valid_acc: 0.291 best_acc: 0.291 	

Epoch 8 train_loss: 2.090 train_f1: 0.061 	

Validation 8 valid_acc: 0.284 best_acc: 0.291 	

Epoch 9 train_loss: 2.084 train_f1: 0.063 	

Validation 9 valid_acc: 0.306 best_acc: 0.306 	

Epoch 10 train_loss: 2.061 train_f1: 0.062 	

Validation 10 valid_acc: 0.299 best_acc: 0.306 	

Epoch 11 train_loss: 2.0


Validation 86 valid_acc: 0.571 best_acc: 0.587 	

Epoch 87 train_loss: 1.417 train_f1: 0.423 	

Validation 87 valid_acc: 0.575 best_acc: 0.587 	

Epoch 88 train_loss: 1.427 train_f1: 0.418 	

Validation 88 valid_acc: 0.581 best_acc: 0.587 	

Epoch 89 train_loss: 1.424 train_f1: 0.425 	

Validation 89 valid_acc: 0.594 best_acc: 0.594 	

Epoch 90 train_loss: 1.419 train_f1: 0.425 	

Validation 90 valid_acc: 0.590 best_acc: 0.594 	

Epoch 91 train_loss: 1.395 train_f1: 0.431 	

Validation 91 valid_acc: 0.588 best_acc: 0.594 	

Epoch 92 train_loss: 1.404 train_f1: 0.434 	

Validation 92 valid_acc: 0.590 best_acc: 0.594 	

Epoch 93 train_loss: 1.394 train_f1: 0.439 	

Validation 93 valid_acc: 0.607 best_acc: 0.607 	

Epoch 94 train_loss: 1.388 train_f1: 0.446 	

Validation 94 valid_acc: 0.601 best_acc: 0.607 	

Epoch 95 train_loss: 1.389 train_f1: 0.436 	

Validation 95 valid_acc: 0.597 best_acc: 0.607 	

Epoch 96 train_loss: 1.378 train_f1: 0.445 	

Validation 96 valid_acc: 0.606 best_acc


Validation 170 valid_acc: 0.697 best_acc: 0.703 	

Epoch 171 train_loss: 1.000 train_f1: 0.646 	

Validation 171 valid_acc: 0.683 best_acc: 0.703 	

Epoch 172 train_loss: 0.986 train_f1: 0.646 	

Validation 172 valid_acc: 0.686 best_acc: 0.703 	

Epoch 173 train_loss: 0.975 train_f1: 0.645 	

Validation 173 valid_acc: 0.697 best_acc: 0.703 	

Epoch 174 train_loss: 0.975 train_f1: 0.655 	

Validation 174 valid_acc: 0.697 best_acc: 0.703 	

Epoch 175 train_loss: 0.951 train_f1: 0.661 	

Validation 175 valid_acc: 0.698 best_acc: 0.703 	

Epoch 176 train_loss: 0.952 train_f1: 0.658 	

Validation 176 valid_acc: 0.705 best_acc: 0.705 	

Epoch 177 train_loss: 0.957 train_f1: 0.658 	

Validation 177 valid_acc: 0.705 best_acc: 0.705 	

Epoch 178 train_loss: 0.958 train_f1: 0.657 	

Validation 178 valid_acc: 0.699 best_acc: 0.705 	

Epoch 179 train_loss: 0.936 train_f1: 0.668 	

Validation 179 valid_acc: 0.705 best_acc: 0.705 	

Epoch 180 train_loss: 0.948 train_f1: 0.665 	

Validation 180 vali


Validation 254 valid_acc: 0.745 best_acc: 0.760 	

Epoch 255 train_loss: 0.666 train_f1: 0.777 	

Validation 255 valid_acc: 0.762 best_acc: 0.762 	

Epoch 256 train_loss: 0.681 train_f1: 0.777 	

Validation 256 valid_acc: 0.751 best_acc: 0.762 	

Epoch 257 train_loss: 0.652 train_f1: 0.781 	

Validation 257 valid_acc: 0.753 best_acc: 0.762 	

Epoch 258 train_loss: 0.663 train_f1: 0.781 	

Validation 258 valid_acc: 0.762 best_acc: 0.762 	

Epoch 259 train_loss: 0.666 train_f1: 0.778 	

Validation 259 valid_acc: 0.760 best_acc: 0.762 	

Epoch 260 train_loss: 0.658 train_f1: 0.777 	

Validation 260 valid_acc: 0.765 best_acc: 0.765 	

Epoch 261 train_loss: 0.654 train_f1: 0.778 	

Validation 261 valid_acc: 0.765 best_acc: 0.765 	

Epoch 262 train_loss: 0.648 train_f1: 0.785 	

Validation 262 valid_acc: 0.768 best_acc: 0.768 	

Epoch 263 train_loss: 0.686 train_f1: 0.774 	

Validation 263 valid_acc: 0.769 best_acc: 0.769 	

Epoch 264 train_loss: 0.654 train_f1: 0.779 	

Validation 264 vali


Validation 338 valid_acc: 0.774 best_acc: 0.775 	

Epoch 339 train_loss: 0.468 train_f1: 0.844 	

Validation 339 valid_acc: 0.763 best_acc: 0.775 	

Epoch 340 train_loss: 0.502 train_f1: 0.835 	

Validation 340 valid_acc: 0.764 best_acc: 0.775 	

Epoch 341 train_loss: 0.465 train_f1: 0.847 	

Validation 341 valid_acc: 0.751 best_acc: 0.775 	

Epoch 342 train_loss: 0.468 train_f1: 0.842 	

Validation 342 valid_acc: 0.772 best_acc: 0.775 	

Epoch 343 train_loss: 0.477 train_f1: 0.842 	

Validation 343 valid_acc: 0.769 best_acc: 0.775 	

Epoch 344 train_loss: 0.469 train_f1: 0.844 	

Validation 344 valid_acc: 0.766 best_acc: 0.775 	

Epoch 345 train_loss: 0.477 train_f1: 0.845 	

Validation 345 valid_acc: 0.754 best_acc: 0.775 	

Epoch 346 train_loss: 0.465 train_f1: 0.843 	

Validation 346 valid_acc: 0.764 best_acc: 0.775 	

Epoch 347 train_loss: 0.457 train_f1: 0.851 	

Validation 347 valid_acc: 0.763 best_acc: 0.775 	

Epoch 348 train_loss: 0.466 train_f1: 0.844 	

Validation 348 vali

...


Validation 590 valid_acc: 0.762 best_acc: 0.782 	

Epoch 591 train_loss: 0.149 train_f1: 0.950 	

Validation 591 valid_acc: 0.765 best_acc: 0.782 	

Epoch 592 train_loss: 0.150 train_f1: 0.952 	

Validation 592 valid_acc: 0.762 best_acc: 0.782 	

Epoch 593 train_loss: 0.150 train_f1: 0.946 	

Validation 593 valid_acc: 0.769 best_acc: 0.782 	

Epoch 594 train_loss: 0.164 train_f1: 0.946 	

Validation 594 valid_acc: 0.751 best_acc: 0.782 	

Epoch 595 train_loss: 0.140 train_f1: 0.952 	

Validation 595 valid_acc: 0.762 best_acc: 0.782 	

Epoch 596 train_loss: 0.143 train_f1: 0.952 	

Validation 596 valid_acc: 0.766 best_acc: 0.782 	

Epoch 597 train_loss: 0.137 train_f1: 0.956 	

Validation 597 valid_acc: 0.742 best_acc: 0.782 	

Epoch 598 train_loss: 0.151 train_f1: 0.948 	

Validation 598 valid_acc: 0.745 best_acc: 0.782 	

Epoch 599 train_loss: 0.153 train_f1: 0.949 	

Validation 599 valid_acc: 0.754 best_acc: 0.782 	

Epoch 600 train_loss: 0.143 train_f1: 0.952 	

Validation 600 vali


Validation 674 valid_acc: 0.740 best_acc: 0.782 	

Epoch 675 train_loss: 0.084 train_f1: 0.974 	

Validation 675 valid_acc: 0.742 best_acc: 0.782 	

Epoch 676 train_loss: 0.091 train_f1: 0.972 	

Validation 676 valid_acc: 0.759 best_acc: 0.782 	

Epoch 677 train_loss: 0.095 train_f1: 0.970 	

Validation 677 valid_acc: 0.759 best_acc: 0.782 	

Epoch 678 train_loss: 0.088 train_f1: 0.971 	

Validation 678 valid_acc: 0.764 best_acc: 0.782 	

Epoch 679 train_loss: 0.112 train_f1: 0.965 	

Validation 679 valid_acc: 0.736 best_acc: 0.782 	

Epoch 680 train_loss: 0.093 train_f1: 0.970 	

Validation 680 valid_acc: 0.750 best_acc: 0.782 	

Epoch 681 train_loss: 0.108 train_f1: 0.966 	

Validation 681 valid_acc: 0.765 best_acc: 0.782 	

Epoch 682 train_loss: 0.098 train_f1: 0.969 	

Validation 682 valid_acc: 0.751 best_acc: 0.782 	

Epoch 683 train_loss: 0.107 train_f1: 0.963 	

Validation 683 valid_acc: 0.753 best_acc: 0.782 	

Epoch 684 train_loss: 0.087 train_f1: 0.969 	

Validation 684 vali


Validation 758 valid_acc: 0.761 best_acc: 0.782 	

Epoch 759 train_loss: 0.074 train_f1: 0.976 	

Validation 759 valid_acc: 0.754 best_acc: 0.782 	

Epoch 760 train_loss: 0.078 train_f1: 0.974 	

Validation 760 valid_acc: 0.764 best_acc: 0.782 	

Epoch 761 train_loss: 0.071 train_f1: 0.978 	

Validation 761 valid_acc: 0.747 best_acc: 0.782 	

Epoch 762 train_loss: 0.075 train_f1: 0.975 	

Validation 762 valid_acc: 0.750 best_acc: 0.782 	

Epoch 763 train_loss: 0.072 train_f1: 0.976 	

Validation 763 valid_acc: 0.762 best_acc: 0.782 	

Epoch 764 train_loss: 0.059 train_f1: 0.982 	

Validation 764 valid_acc: 0.752 best_acc: 0.782 	

Epoch 765 train_loss: 0.062 train_f1: 0.981 	

Validation 765 valid_acc: 0.757 best_acc: 0.782 	

Epoch 766 train_loss: 0.074 train_f1: 0.976 	

Validation 766 valid_acc: 0.758 best_acc: 0.782 	

Epoch 767 train_loss: 0.064 train_f1: 0.981 	

Validation 767 valid_acc: 0.746 best_acc: 0.782 	

Epoch 768 train_loss: 0.075 train_f1: 0.977 	

Validation 768 vali


Validation 842 valid_acc: 0.744 best_acc: 0.782 	

Epoch 843 train_loss: 0.048 train_f1: 0.984 	

Validation 843 valid_acc: 0.754 best_acc: 0.782 	

Epoch 844 train_loss: 0.053 train_f1: 0.982 	

Validation 844 valid_acc: 0.750 best_acc: 0.782 	

Epoch 845 train_loss: 0.051 train_f1: 0.984 	

Validation 845 valid_acc: 0.753 best_acc: 0.782 	

Epoch 846 train_loss: 0.046 train_f1: 0.985 	

Validation 846 valid_acc: 0.754 best_acc: 0.782 	

Epoch 847 train_loss: 0.039 train_f1: 0.987 	

Validation 847 valid_acc: 0.754 best_acc: 0.782 	

Epoch 848 train_loss: 0.058 train_f1: 0.980 	

Validation 848 valid_acc: 0.746 best_acc: 0.782 	

Epoch 849 train_loss: 0.052 train_f1: 0.981 	

Validation 849 valid_acc: 0.758 best_acc: 0.782 	

Epoch 850 train_loss: 0.046 train_f1: 0.986 	

Validation 850 valid_acc: 0.762 best_acc: 0.782 	

Epoch 851 train_loss: 0.055 train_f1: 0.983 	

Validation 851 valid_acc: 0.748 best_acc: 0.782 	

Epoch 852 train_loss: 0.058 train_f1: 0.982 	

Validation 852 vali


Validation 926 valid_acc: 0.747 best_acc: 0.782 	

Epoch 927 train_loss: 0.040 train_f1: 0.988 	

Validation 927 valid_acc: 0.743 best_acc: 0.782 	

Epoch 928 train_loss: 0.041 train_f1: 0.987 	

Validation 928 valid_acc: 0.745 best_acc: 0.782 	

Epoch 929 train_loss: 0.034 train_f1: 0.989 	

Validation 929 valid_acc: 0.751 best_acc: 0.782 	

Epoch 930 train_loss: 0.042 train_f1: 0.985 	

Validation 930 valid_acc: 0.758 best_acc: 0.782 	

Epoch 931 train_loss: 0.036 train_f1: 0.988 	

Validation 931 valid_acc: 0.754 best_acc: 0.782 	

Epoch 932 train_loss: 0.037 train_f1: 0.987 	

Validation 932 valid_acc: 0.735 best_acc: 0.782 	

Epoch 933 train_loss: 0.044 train_f1: 0.987 	

Validation 933 valid_acc: 0.749 best_acc: 0.782 	

Epoch 934 train_loss: 0.036 train_f1: 0.988 	

Validation 934 valid_acc: 0.758 best_acc: 0.782 	

Epoch 935 train_loss: 0.039 train_f1: 0.989 	

Validation 935 valid_acc: 0.746 best_acc: 0.782 	

Epoch 936 train_loss: 0.045 train_f1: 0.984 	

Validation 936 vali